In [ ]:
%%capture

pip install mne

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import mne
import pywt

In [ ]:
%%capture
# folder = 'E:/Data WareHouse/1sec/Epileptic Seizure/equalized'
folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch'
epochs_path = [os.path.join(folder,i) for i in os.listdir(folder) if i[-3:]=='fif']

data= [mne.read_epochs(i).pick_types(eeg=True) for i in epochs_path]
labels = [mne.read_epochs(i).events[:,2] for i in epochs_path]
Y=np.hstack(labels)

In [ ]:
t_f_domain_folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time frequency domain/RAW'
delta_path, theta_path, alpha_path, beta_path = [os.path.join(t_f_domain_folder,i) for i in os.listdir(t_f_domain_folder)]


delta_data= np.load(delta_path)
theta_data= np.load(theta_path)
alpha_data= np.load(alpha_path)
beta_data= np.load(beta_path)

delta_reshape = delta_data.reshape(delta_data.shape[0], -1)
theta_reshape = theta_data.reshape(theta_data.shape[0], -1)
alpha_reshape = alpha_data.reshape(alpha_data.shape[0], -1)
beta_reshape = beta_data.reshape(beta_data.shape[0], -1)



In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier,\
                                StackingClassifier,VotingClassifier,IsolationForest,\
                                RandomForestRegressor
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import CategoricalNB
from xgboost import XGBClassifier
# import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier

from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV,GridSearchCV

In [ ]:
#Scale the data
SS1= StandardScaler()
delta_X = SS1.fit_transform(delta_reshape)
SS2= StandardScaler()
theta_X = SS2.fit_transform(theta_reshape)
SS3= StandardScaler()
alpha_X = SS3.fit_transform(alpha_reshape)
SS4= StandardScaler()
beta_X = SS4.fit_transform(beta_reshape)

In [ ]:
models = {
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'HistGradientBoosting': HistGradientBoostingClassifier(),
    'SVM': SVC(),
    'GradientBoosting': GradientBoostingClassifier(),
    'KNeighbors': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    # 'MultinomialNB': MultinomialNB(),
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'LogisticRegression': LogisticRegression(),
    'AdaBoost': AdaBoostClassifier(),
    # 'Bagging': BaggingClassifier(),
    'ExtraTrees': ExtraTreesClassifier(),
    # 'GaussianProcess': GaussianProcessClassifier(kernel=RBF()),
    # 'CategoricalNB': CategoricalNB(),
    # 'LightGBM': lgb.LGBMClassifier(),
    # 'RandomForestRegressor': RandomForestRegressor(),  # Note: This is a regressor, not a classifier
    # 'IsolationForest': IsolationForest()  # Note: This is typically used for anomaly detection
    # Add any additional models here
}

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef
from sklearn.model_selection import KFold
import pandas as pd

def evaluate_model(model, X, y, n_splits):
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=42)
    accuracies, precisions, recalls,f1s,cm,specificitys, sensitivitys,roc_aucs,kappas,mccs= [], [], [],[],[],[],[],[],[],[]

    for train_idx, test_idx in kf.split(X):
      # take the location of the splited data and access it by the index

      X_train, y_train = (X.iloc[train_idx], y.iloc[train_idx]) if isinstance(X, pd.DataFrame) else (X[train_idx], y[train_idx])
      X_test, y_test = (X.iloc[test_idx], y.iloc[test_idx]) if isinstance(X, pd.DataFrame) else (X[test_idx], y[test_idx])
      # train the model
      model.fit(X_train, y_train)
      # make prediction
      y_pred = model.predict(X_test)

      # evaluation scores
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred,zero_division=1)
      recall = recall_score(y_test, y_pred)
      f1= f1_score(y_test, y_pred)
      cms = confusion_matrix(y_test, y_pred)  # Calculate confusion matrix
      roc_auc=roc_auc_score(y_test, y_pred)

      # calculate the specificity and sensitivity
      tn, fp, fn, tp = cms.ravel()
      specificity = tn / (tn + fp)
      sensitivity = tp / (tp + fn)
      # Calculate Cohen's Kappa
      kappa = cohen_kappa_score(y_test, y_pred)
      mcc = matthews_corrcoef(y_test, y_pred)





      # append all the results
      accuracies.append(accuracy)
      precisions.append(precision)
      recalls.append(recall)
      f1s.append(f1)
      cm.append(cms)  # Add confusion matrix to list
      specificitys.append(specificity) #add specificity score
      sensitivitys.append(sensitivity) #add sensitivity scores to the list
      roc_aucs.append(roc_auc) #add roc_auc score to the list
      kappas.append(kappa)
      mccs.append(mcc)


    return (format(sum(accuracies)/n_splits, '.3f'), format(sum(precisions)/n_splits, '.3f'),
            format(sum(recalls)/n_splits, '.3f'),format(sum(f1s)/n_splits, '.3f'),format(sum(roc_aucs)/n_splits, '.3f'), sum(cm)/n_splits ,format(sum(specificitys)/n_splits, '.3f'),
            format(sum(sensitivitys)/n_splits, '.3f'),format(sum(kappas)/n_splits, '.3f'),format(sum(mccs)/n_splits, '.3f'))




def modelApplied(Model,X_train , y_train, n_splits=5,MN='model_name',model_result = pd.DataFrame()):
  accuracy, precision, recall,f1,roc_auc,cm,specificity,sensitivity,kappa,mcc = evaluate_model(Model,X_train , y_train, n_splits)

  new_row=[MN,accuracy,precision,recall,f1,roc_auc,specificity, sensitivity,kappa,mcc]

  model_result.loc[len(model_result)]=new_row
  print(new_row)
  print('DONE!!')

In [ ]:
alpha_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>alpha<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, beta_X, Y, MN=k,model_result= alpha_result)

#Model Optimization:

In [ ]:
# import xgboost as xgb
# # Initialize XGBClassifier
# xgb_model = RandomForestClassifier()

# # Define the parameter grid
# param_grid = {
#     'n_estimators': np.arange(100,1000,100),
#     # 'learning_rate': [0.01, 0.05, 0.1, 0.2],
#     # 'max_depth': [3, 4, 5, 6, 7],
#     # 'min_child_weight': [1, 2, 3, 4],
#     # 'gamma': [0, 0.1, 0.2, 0.3],
#     # 'colsample_bytree': [0.3, 0.4, 0.5, 0.7]
# }

# # Initialize RandomizedSearchCV
# randomized_cv = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, scoring='accuracy', cv=3, random_state=42)

# # Fit the model
# randomized_cv.fit(beta_X, Y)

# # Print the best parameters and highest accuracy
# print("Best parameters found: ", randomized_cv.best_params_)
# print("Best accuracy found: ", randomized_cv.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# import xgboost as xgb
# # Initialize XGBClassifier
# xgb_model = xgb.XGBClassifier(n_estimators=400,)


# all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

# print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> beta <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

# modelApplied(xgb_model, beta_X, Y, MN='XGB',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> beta <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGB', '0.924', '0.945', '0.899', '0.922', '0.924', '0.948', '0.899', '0.847', '0.848']
DONE!!


In [ ]:
import xgboost as xgb
# Initialize XGBClassifier
xgb_model = xgb.XGBClassifier(n_estimators=450,)


all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> beta <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(xgb_model, beta_X, Y, MN='XGB',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> beta <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGB', '0.924', '0.945', '0.900', '0.922', '0.924', '0.947', '0.900', '0.847', '0.848']
DONE!!


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> beta <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ['XGB', '0.924', '0.945', '0.900', '0.922', '0.924', '0.947', '0.900', '0.847', '0.848']
# DONE!!

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Initialize XGBClassifier
RF_model = RandomForestClassifier(n_estimators=150,criterion = 'entropy',max_features = None
                                  )
all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(RF_model, beta_X, Y, MN='RF',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['RF', '0.901', '0.929', '0.868', '0.897', '0.901', '0.933', '0.868', '0.802', '0.803']
DONE!!


In [ ]:
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['RF', '0.903', '0.933', '0.869', '0.900', '0.903', '0.937', '0.869', '0.806', '0.808']
DONE!!
n_estimators=150,criterion = 'entropy',max_features = None

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
# Initialize XGBClassifier
HGB_model = HistGradientBoostingClassifier(interaction_cst= 'pairwise')
all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(HGB_model, beta_X, Y, MN='HGB',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['HGB', '0.898', '0.924', '0.867', '0.894', '0.898', '0.929', '0.867', '0.796', '0.797']
DONE!!


In [ ]:
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['HGB', '0.921', '0.946', '0.892', '0.918', '0.921', '0.949', '0.892', '0.842', '0.843']
DONE!!
20